In [54]:
# 압축파일 폴더별 정리
import os
import zipfile
path = 'D:/Workspace/04.Project/project_02/data/label/Training' # 폴더 경로 설정
file_list = os.listdir(path) # 압축파일 이름 리스트화

for file_name in file_list[2:]: # 압축파일 리스트화한걸 이름으로 하나씩 넘김
      with zipfile.ZipFile(f'{file_name}', 'r') as existing_zip: # 압축파일 경로 설정
        foldername = file_name[:-4] # 폴더 이름 추출
        os.mkdir(foldername) # 폴더 생성
        existing_zip.extractall(foldername) #압축을 풀어 폴더에 넣기

In [123]:
# 라벨링 파일 DF 로 변환 작업
# 작업 진행률 확인 tqdm_notebook 추가
import pandas as pd
import json
from tqdm import tqdm_notebook 
path = 'D:/Workspace/04.Project/project_02/data/label/Training' # 폴더 경로 설정
folder_list = os.listdir(path) # 폴더 이름 리스트화

# 최종 모든 라벨링 데이터 모음 데이터프레임
df_raw = pd.DataFrame(index=range(0,), columns=['image', 'date', 'worker', 'height', 'width', 'task', 'type', 'region',
       'disease', 'crop', 'area', 'grow', 'risk', 'points', 'name']) # 폴더내용 저장할 빈 데이터프레임 생성


       
for folder_name in tqdm_notebook(folder_list):
    print(folder_name)
    file_list = os.listdir(f'{path}/{folder_name}/')
    for file_name in file_list:
        # 각 폴더별 라벨링을 위한 임시 데이터 프레임
        df = pd.DataFrame(index=range(0,), columns=['image', 'date', 'worker', 'height', 'width', 'task', 'type', 'region',
       'disease', 'crop', 'area', 'grow', 'risk', 'points']) # 폴더내용 저장할 빈 데이터프레임 생성
        with open(f'{path}/{folder_name}/{file_name}', 'r') as f:
            json_data = json.load(f)
            tmp = pd.DataFrame(json_data)
            tmp = tmp[['description', 'annotations']][:15]
            tmp['description'][8:15] = tmp['annotations'][8:15] # 데이터프레임 내용 병합
            del(tmp['annotations']) # 필요없는 열 삭제
            tmp = tmp.T # 전치
            df = df.append(tmp) # df에 추가
            df['name'] = folder_name[4:] # 구분을 위한 name 열 생성
            df_raw = df_raw.append(df) # 최종 데이터 프레임에 추가

df_raw

<ipython-input-123-fd3c92ce8677>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for folder_name in tqdm_notebook(folder_list):


  0%|          | 0/20 [00:00<?, ?it/s]

[라벨]고추_0.정상
[라벨]고추_1.질병
[라벨]무_0.정상
[라벨]무_1.질병
[라벨]배추_0.정상
[라벨]배추_1.질병
[라벨]애호박_0.정상
[라벨]애호박_1.질병
[라벨]양배추_0.정상
[라벨]양배추_1.질병
[라벨]오이_0.정상
[라벨]오이_1.질병
[라벨]잎마름병(토마토)_0.정상


In [100]:
# 최종파일 csv 저장
df_raw.to_csv('D:/Workspace/04.Project/project_02/data/train_label.csv', index=False)